In [2]:
# import all necessary libraries
import numpy as np
import cv2
import glob
import math

# variable to store which video we must elaborate, we must insert only numbers 1,2,3 or 4
videoToElaborate = 2


# function to rotate one point around origin for some angles using a depth value always equal to 0 because we change it in the project call function
def rotate(origin, point, angle):
    x = origin[0] + np.cos(angle) * (point[0] - origin[0]) - np.sin(angle) * (point[1] - origin[1])
    y = origin[1] + np.sin(angle) * (point[0] - origin[0]) + np.cos(angle) * (point[1] - origin[1])
    return [x, y, 0]


# load the matrix camera and distortion matrix after the calibration process
dist = np.load("framesVideoCalibration/uncalibrated/dist.npy")
K = np.load("framesVideoCalibration/uncalibrated/K.npy")

# we create a cube 70*70*60 to store all points if they are projected into the object
# i create it with this small dimensions because otherwise the process to project all the points is very long
cube = np.zeros([70, 70, 60])

# get list of frames of video from the uncalibrated folder
listImages = glob.glob("framesVideo" + str(videoToElaborate) + "/uncalibrated/*.jpg")

# temp variable to store for each polygon which vertex is the convex one
temp = 0

# for each image
for image in listImages:

    # ---------------------------------------------------------------------------------------------------- BLACK POLYGON AND WHITE CIRCLES DETECTION

    # open the actual image in color mode
    actual_image = cv2.imread(image, cv2.IMREAD_COLOR)

    # copy of the actual image to use it in the next parts
    copy_actual_image = cv2.imread(image, cv2.IMREAD_COLOR)

    # open the actual image in gray mode
    gray_actual_image = cv2.imread(image, cv2.IMREAD_GRAYSCALE)

    # make binary threshold on gray actual image
    # If pixel intensity is greater than the set threshold, value set to 255 (white), else set to 0 (black)
    _, threshold_gray_actual_image = cv2.threshold(gray_actual_image, 190, 255, cv2.THRESH_BINARY)

    # save the threshold gray actual image in the binarythreshold folder
    cv2.imwrite(image.replace("uncalibrated", "binarythreshold"), threshold_gray_actual_image)

    # detect shape in threshold gray actual image using regions with same colors and intensity
    # RETR_TREE to get a hierarchy of contours
    # CHAIN_APPROX_NONE all the boundary points are stored
    contours, hierarchy = cv2.findContours(image=threshold_gray_actual_image, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)

    # save the index of actual contour
    index_actual_contour = 0

    # ---------------------------------------------------------------------------------------------------- VARIABLES FOR DETECTED POLYGON WITH KNOWN BINARY VALUES

    # variables for detection of combination 100
    contours_100 = []
    approx_100 = []
    points_100 = []
    concave_vertex_100 = []

    # variables for detection of combination 102
    contours_102 = []
    approx_102 = []
    points_102 = []
    concave_vertex_102 = []

    # variables for detection of combination 112
    contours_112 = []
    approx_112 = []
    points_112 = []
    concave_vertex_112 = []

    # variables for detection of combination 110
    contours_110 = []
    approx_110 = []
    points_110 = []
    concave_vertex_110 = []

    # variables for detection of combination 212
    contours_212 = []
    approx_212 = []
    points_212 = []
    concave_vertex_212 = []

    # variables for detection of combination 202
    contours_202 = []
    approx_202 = []
    points_202 = []
    concave_vertex_202 = []

    # variables for detection of combination 200
    contours_200 = []
    approx_200 = []
    points_200 = []
    concave_vertex_200 = []

    # variables for detection of combination 210
    contours_210 = []
    approx_210 = []
    points_210 = []
    concave_vertex_210 = []

    # variables for detection of combination 211
    contours_211 = []
    approx_211 = []
    points_211 = []
    concave_vertex_211 = []

    # variables for detection of combination 201
    contours_201 = []
    approx_201 = []
    points_201 = []
    concave_vertex_201 = []

    # variables for detection of combination 111
    contours_111 = []
    approx_111 = []
    points_111 = []
    concave_vertex_111 = []

    # variables for detection of combination 101
    contours_101 = []
    approx_101 = []
    points_101 = []
    concave_vertex_101 = []

    # ---------------------------------------------------------------------------------------------------- BLACK POLYGON AND WHITE CIRCLES DETECTION

    # for each contour in threshold_gray_actual_image
    for contour in contours:

        # calculate the area of the actual region
        area_actual_region = cv2.contourArea(contour)

        # approximate the shape of the actual polygon
        approx = cv2.approxPolyDP(contour, 0.01 * cv2.arcLength(contour, True), True)

        # if the area of the actual region is bigger than a minimum value
        if area_actual_region > 200:

            # if we have 5 edges in the actual region
            if len(approx) == 5:

                # calculate the perimeter of the contour
                perimeter = cv2.arcLength(approx, True)

                # calculate the average of one edge
                avg_perimeter = perimeter / len(approx)

                # save index of concave vertex
                first_edge_concave_vertex_clockwise = 0

                # initialize the list of children of actual polygon as empty
                children_contours = []

                # we know that contours have form [next, previous, first child, parent]
                children = hierarchy[0][index_actual_contour][2]

                # while there are other children
                while children != -1:
                    # append actual children to the previous list
                    children_contours.append(children)

                    # obtain the right contour
                    children = hierarchy[0][children][0]

                # area children contours
                area_children_contours = []

                # for each children contours
                for children_contour in children_contours:
                    # calculate area actual children
                    area_actual_children = cv2.contourArea(contours[children_contour])

                    # save the actual area
                    area_children_contours.append(area_actual_children)

                # temp array to store the children contours
                temp_children_contours = children_contours.copy()

                # initialize as empty array the children contours
                children_contours = []

                # for each area contour
                for i in range(0, len(area_children_contours)):

                    # if the actual area contour is greater than 20 and so it is a valid area, because there can be some children with a smaller to 0 area
                    # and so they are not valid polygon
                    if area_children_contours[i] > 20:
                        # save the actual children contour as valid
                        children_contours.append(temp_children_contours[i])

                # convert the children contours list in numpy array
                children_contours = np.array(children_contours)

                # if there are children in actual contour
                if len(children_contours) > 0:

                    # at this point we skip the detected polygon but where these polygons have not white circle

                    # for each edge
                    for i in range(0, len(approx)):

                        # calculate the distance between the previous and the next one
                        previous = math.dist([approx[i][0][0], approx[i][0][1]], [approx[(i - 1) % len(approx)][0][0], approx[(i - 1) % len(approx)][0][1]])
                        next = math.dist([approx[i][0][0], approx[i][0][1]], [approx[(i + 1) % len(approx)][0][0], approx[(i + 1) % len(approx)][0][1]])

                        # if they are both less than average edge
                        if previous < avg_perimeter and next < avg_perimeter:

                            # this is the concave vertex
                            first_edge_concave_vertex_clockwise = i

                            #if len(children_contours) == 5:
                            #temp = first_edge_concave_vertex_clockwise

                            # calculate the edge of the outer part of the polygon
                            second = approx[(i + 2) % len(approx)][0]
                            third = approx[(i + 3) % len(approx)][0]

                            # calculate the middle point of the previous edge
                            half_second_third = [(second[0] + third[0]) // 2, (second[1] + third[1]) // 2]

                            # calculate the middle point between the previous middle point and the concave vertex
                            half_point_polygon = [(approx[first_edge_concave_vertex_clockwise][0][0] + half_second_third[0]) // 2, (approx[first_edge_concave_vertex_clockwise][0][1] + half_second_third[1]) // 2]

                            # draw a circle in the previous middle point
                            cv2.circle(actual_image, (half_point_polygon[0], half_point_polygon[1]), 1, (0, 0, 255), -1)

                            # now draw circles of different color from the concave vertex in clockwise
                            for j in range(0, len(approx)):

                                # if it is the concave vertex
                                if j == 0:

                                    # red circle
                                    actual_image = cv2.circle(actual_image, (int(approx[(first_edge_concave_vertex_clockwise + j) % len(approx)][0][0]), int(approx[(first_edge_concave_vertex_clockwise + j) % len(approx)][0][1])), radius=4, color=(0, 0, 255), thickness=-1)

                                # one vertex after the concave vertex
                                elif j == 1:

                                    # green circle
                                    actual_image = cv2.circle(actual_image, (int(approx[(first_edge_concave_vertex_clockwise + j) % len(approx)][0][0]), int(approx[(first_edge_concave_vertex_clockwise + j) % len(approx)][0][1])), radius=4, color=(0, 255, 0), thickness=-1)

                                # two vertex after the concave vertex
                                elif j == 2:

                                    # blue circle
                                    actual_image = cv2.circle(actual_image, (int(approx[(first_edge_concave_vertex_clockwise + j) % len(approx)][0][0]), int(approx[(first_edge_concave_vertex_clockwise + j) % len(approx)][0][1])), radius=4, color=(255, 0, 0), thickness=-1)

                                # three vertex after the concave vertex
                                elif j == 3:

                                    # yellow circle
                                    actual_image = cv2.circle(actual_image, (int(approx[(first_edge_concave_vertex_clockwise + j) % len(approx)][0][0]), int(approx[(first_edge_concave_vertex_clockwise + j) % len(approx)][0][1])), radius=4, color=(0, 255, 255), thickness=-1)

                                # four vertex after the concave vertex
                                elif j == 4:

                                    # pink circle
                                    actual_image = cv2.circle(actual_image, (int(approx[(first_edge_concave_vertex_clockwise + j) % len(approx)][0][0]), int(approx[(first_edge_concave_vertex_clockwise + j) % len(approx)][0][1])), radius=4, color=(255, 0, 255), thickness=-1)

                            # we can exit from the for of approx edges because we have draw all the vertices of actual polygon
                            break

                    # we can draw red contours of the actual region but in the colored image
                    cv2.drawContours(actual_image, [contour], 0, (0, 0, 255), 1)

                    # ---------------------------------------------------------------------------------------------------- FINISH BLACK POLYGON AND WHITE CIRCLES DETECTION

                    # ---------------------------------------------------------------------------------------------------- DETECTION OF INNER CENTER AND OUTER POINT

                    # create a empty list to store all important points for each polygon
                    points = []

                    # calculate the edge of the outer polygon
                    second = approx[(first_edge_concave_vertex_clockwise + 2) % len(approx)][0]
                    third = approx[(first_edge_concave_vertex_clockwise + 3) % len(approx)][0]

                    # calculate the middle point of the previous edge
                    half_second_third = [(second[0] + third[0]) // 2, (second[1] + third[1]) // 2]

                    # calculate the middle point between the previous middle point and the concave vertex
                    half_point_polygon = [(approx[first_edge_concave_vertex_clockwise][0][0] + half_second_third[0]) // 2, (approx[first_edge_concave_vertex_clockwise][0][1] + half_second_third[1]) // 2]

                    # append to the list the outer point
                    points.append(half_second_third)

                    # append to the list the middle point of the middle edge in the polygon
                    points.append(half_point_polygon)

                    # append to the list the inner point
                    points.append([approx[first_edge_concave_vertex_clockwise][0][0], approx[first_edge_concave_vertex_clockwise][0][1]])

                    # for each children contour
                    for single_children in children_contours:

                        # draw contours of actual children
                        cv2.drawContours(actual_image, contours, single_children, (255, 0, 0), 1)

                        # calculate the center of actual contour
                        M = cv2.moments(contours[single_children])
                        if M["m00"] != 0:
                            cX = int(M["m10"] / M["m00"])
                            cY = int(M["m01"] / M["m00"])

                            # append to the list the central point of actual contour
                            points.append([cX, cY])

                    # save the coordinates of the outer point
                    x = points[0][0]
                    y = points[0][1]

                    # order list of points from outer to inner points through a line 2d
                    points.sort(key=lambda p: (p[0] - x) ** 2 + (p[1] - y) ** 2)

                    # counter for outer points
                    counter_outer_points = 0

                    # counter for central points
                    counter_central_points = 0

                    # counter for inner points
                    counter_inner_points = 0

                    # boolean to check if the central point is found
                    check_center_point_found = False

                    # for each point of polygon (no first because is the outer point, no last one because is the inner point)
                    for i in range(1, len(points) - 1):

                        # if the actual point is the central point of the polygon and is the first time that i found the central point
                        if points[i][0] == half_point_polygon[0] and points[i][1] == half_point_polygon[1] and not check_center_point_found:

                            # center point of the polygon found
                            check_center_point_found = True

                        # if it is a valid point to check
                        else:

                            # index for the correct contour
                            index_correct_contour = 0

                            # find the contour associated with the actual point
                            for single_children in children_contours:

                                # calculate the center of actual contour
                                M = cv2.moments(contours[single_children])
                                if M["m00"] != 0:
                                    cX = int(M["m10"] / M["m00"])
                                    cY = int(M["m01"] / M["m00"])

                                    # if the coordinates are correct
                                    if cX == points[i][0] and cY == points[i][1]:
                                        cv2.circle(actual_image, (cX, cY), 3, (40, 40, 40), -1)

                                        # save the correct index for the contour
                                        index_correct_contour = single_children

                                        # exit from the for loop because the correct contour is found
                                        break

                            # create correct data format to use forward the pointPolygonTest function
                            list_array_points = contours[index_correct_contour].tolist()
                            my_tuple = (int(half_point_polygon[0]), int(half_point_polygon[1]))

                            # if the center point is not just found
                            if not check_center_point_found:

                                # if the center point is in the actual contour point

                                if cv2.pointPolygonTest(np.array(list_array_points), my_tuple, False) == 1:

                                    # increment central points
                                    counter_central_points = counter_central_points + 1

                                # if the center point is not in the actual contour point
                                else:

                                    # increment outer points
                                    counter_outer_points = counter_outer_points + 1

                            # if the center point is found
                            else:

                                # if the center point is in the actual contour point
                                if cv2.pointPolygonTest(np.array(list_array_points), my_tuple, False) == 1:

                                    # increment central points
                                    counter_central_points = counter_central_points + 1

                                # if the center point is not in the actual contour point
                                else:

                                    # increment inner points
                                    counter_inner_points = counter_inner_points + 1

                    if counter_outer_points == 2 and counter_central_points == 1 and counter_inner_points == 2:
                        # save the concave vertex
                        concave_vertex_212.append(first_edge_concave_vertex_clockwise)

                        approx_212.append(approx)
                        contours_212.append([contour])
                        points_212.append(points)

                    if counter_outer_points == 2 and counter_central_points == 0 and counter_inner_points == 2:
                        # save the concave vertex
                        concave_vertex_202.append(first_edge_concave_vertex_clockwise)

                        approx_202.append(approx)
                        contours_202.append([contour])
                        points_202.append(points)

                    if counter_outer_points == 2 and counter_central_points == 0 and counter_inner_points == 0:
                        # save the concave vertex
                        concave_vertex_200.append(first_edge_concave_vertex_clockwise)

                        approx_200.append(approx)
                        contours_200.append([contour])
                        points_200.append(points)

                    if counter_outer_points == 2 and counter_central_points == 1 and counter_inner_points == 0:
                        # save the concave vertex
                        concave_vertex_210.append(first_edge_concave_vertex_clockwise)

                        approx_210.append(approx)
                        contours_210.append([contour])
                        points_210.append(points)

                    if counter_outer_points == 1 and counter_central_points == 0 and counter_inner_points == 0:
                        contours_100.append([contour])
                        approx_100.append(approx)
                        points_100.append(points)

                        # save the concave vertex
                        concave_vertex_100.append(first_edge_concave_vertex_clockwise)

                    if counter_outer_points == 1 and counter_central_points == 0 and counter_inner_points == 2:
                        contours_102.append([contour])
                        approx_102.append(approx)
                        points_102.append(points)

                        # save the concave vertex
                        concave_vertex_102.append(first_edge_concave_vertex_clockwise)

                    if counter_outer_points == 1 and counter_central_points == 1 and counter_inner_points == 0:
                        contours_110.append([contour])
                        approx_110.append(approx)
                        points_110.append(points)

                        # save the concave vertex
                        concave_vertex_110.append(first_edge_concave_vertex_clockwise)

                    if counter_outer_points == 1 and counter_central_points == 1 and counter_inner_points == 2:
                        contours_112.append([contour])
                        approx_112.append(approx)
                        points_112.append(points)

                        # save the concave vertex
                        concave_vertex_112.append(first_edge_concave_vertex_clockwise)

                    if counter_outer_points == 2 and counter_central_points == 1 and counter_inner_points == 1:
                        contours_211.append([contour])
                        approx_211.append(approx)
                        points_211.append(points)

                        # save the concave vertex
                        concave_vertex_211.append(first_edge_concave_vertex_clockwise)

                    if counter_outer_points == 2 and counter_central_points == 0 and counter_inner_points == 1:
                        contours_201.append([contour])
                        approx_201.append(approx)
                        points_201.append(points)

                        # save the concave vertex
                        concave_vertex_201.append(first_edge_concave_vertex_clockwise)

                    if counter_outer_points == 1 and counter_central_points == 1 and counter_inner_points == 1:
                        contours_111.append([contour])
                        approx_111.append(approx)
                        points_111.append(points)

                        # save the concave vertex
                        concave_vertex_111.append(first_edge_concave_vertex_clockwise)

                    if counter_outer_points == 1 and counter_central_points == 0 and counter_inner_points == 1:
                        contours_101.append([contour])
                        approx_101.append(approx)
                        points_101.append(points)

                        # save the concave vertex
                        concave_vertex_101.append(first_edge_concave_vertex_clockwise)

                    # ---------------------------------------------------------------------------------------------------- FINISH DETECTION OF INNER CENTER AND OUTER POINT

        # increment the index of actual contour
        index_actual_contour = index_actual_contour + 1

    # ---------------------------------------------------------------------------------------------------- LIST OF 2D AND 3D POINTS FOR THE NEXT PROJECTION PROCESS

    # initialize the list for 2d point as empty
    list_2d_points = []

    # initialize the list for 3d point as empty
    list_3d_points = []

    # if there is one 212 in the image
    if len(approx_212) > 0:
        # reorder array of points using the concave vertex in the first position
        approx_212 = np.roll(approx_212[0], 2 * (len(approx_212[0]) - concave_vertex_212[0]))

        # add points of 212 to 2d points list
        list_2d_points.extend(approx_212)

        # add points of 212 to 3d points list
        list_3d_points.extend([[70, 0, 0], [65, 5, 0], [98, 5, 0], [98, -5, 0], [65, -5, 0]])

        cv2.drawContours(actual_image, contours_212[0], 0, (0, 0, 255), 3)  # red

    # if there is one 202 in the image
    if len(approx_202) > 0:
        # reorder array of points using the concave vertex in the first position
        approx_202 = np.roll(approx_202[0], 2 * (len(approx_202[0]) - concave_vertex_202[0]))

        # add points of 202 to 2d points list
        list_2d_points.extend(approx_202)

        # add points of 202 to 3d points list
        list_3d_points.extend([rotate((0, 0), (70, 0), 20 * np.pi / 12), rotate((0, 0), (65, 5), 20 * np.pi / 12), rotate((0, 0), (98, 5), 20 * np.pi / 12), rotate((0, 0), (98, -5), 20 * np.pi / 12), rotate((0, 0), (65, -5), 20 * np.pi / 12)])

        cv2.drawContours(actual_image, contours_202[0], 0, (0, 255, 0), 3)  # green

    # if there is one 200 in the image
    if len(approx_200) > 0:
        # reorder array of points using the concave vertex in the first position
        approx_200 = np.roll(approx_200[0], 2 * (len(approx_200[0]) - concave_vertex_200[0]))

        # add points of 200 to 2d points list
        list_2d_points.extend(approx_200)

        # add points of 200 to 3d points list
        list_3d_points.extend([rotate((0, 0), (70, 0), 17 * np.pi / 12), rotate((0, 0), (65, 5), 17 * np.pi / 12), rotate((0, 0), (98, 5), 17 * np.pi / 12), rotate((0, 0), (98, -5), 17 * np.pi / 12), rotate((0, 0), (65, -5), 17 * np.pi / 12)])

        cv2.drawContours(actual_image, contours_200[0], 0, (255, 0, 0), 3)  # blue

    # if there is one 210 in the image
    if len(approx_210) > 0:
        # reorder array of points using the concave vertex in the first position
        approx_210 = np.roll(approx_210[0], 2 * (len(approx_210[0]) - concave_vertex_210[0]))

        # add points of 210 to 2d points list
        list_2d_points.extend(approx_210)

        # add points of 210 to 3d points list
        list_3d_points.extend([rotate((0, 0), (70, 0), -3 * np.pi / 12), rotate((0, 0), (65, 5), -3 * np.pi / 12), rotate((0, 0), (98, 5), -3 * np.pi / 12), rotate((0, 0), (98, -5), -3 * np.pi / 12), rotate((0, 0), (65, -5), -3 * np.pi / 12)])

        cv2.drawContours(actual_image, contours_210[0], 0, (128, 0, 128), 3)  # purple

    # if there are two 100 in the image
    if len(points_100) == 2:

        # reorder array of points using the concave vertex in the first position
        approx_100[0] = np.roll(approx_100[0], 2 * (len(approx_100[0]) - concave_vertex_100[0]))
        approx_100[1] = np.roll(approx_100[1], 2 * (len(approx_100[1]) - concave_vertex_100[1]))

        # add points of 100 to 3d points list
        list_3d_points.extend([rotate((0, 0), (70, 0), 9 * np.pi / 12), rotate((0, 0), (65, 5), 9 * np.pi / 12), rotate((0, 0), (98, 5), 9 * np.pi / 12), rotate((0, 0), (98, -5), 9 * np.pi / 12), rotate((0, 0), (65, -5), 9 * np.pi / 12)])
        list_3d_points.extend([rotate((0, 0), (70, 0), 1 * np.pi / 12), rotate((0, 0), (65, 5), 1 * np.pi / 12), rotate((0, 0), (98, 5), 1 * np.pi / 12), rotate((0, 0), (98, -5), 1 * np.pi / 12), rotate((0, 0), (65, -5), 1 * np.pi / 12)])

        # calculate the distance between their two points
        first = (points_100[0][0][0] - points_100[0][1][0]) ** 2 + (points_100[0][0][1] - points_100[0][1][1]) ** 2
        second = (points_100[1][0][0] - points_100[1][1][0]) ** 2 + (points_100[1][0][1] - points_100[1][1][1]) ** 2

        cv2.drawContours(actual_image, contours_100[0], 0, (0, 75, 150), 3)  # brown
        cv2.drawContours(actual_image, contours_100[1], 0, (0, 75, 150), 3)

        # if first distance is greater than the second one
        if first > second:

            # add points of 100 to 2d points list
            list_2d_points.extend(approx_100[1])
            list_2d_points.extend(approx_100[0])

        else:

            # add points of 100 to 2d points list
            list_2d_points.extend(approx_100[0])
            list_2d_points.extend(approx_100[1])

    # if there are two 102 in the image
    if len(points_102) == 2:

        # reorder array of points using the concave vertex in the first position
        approx_102[0] = np.roll(approx_102[0], 2 * (len(approx_102[0]) - concave_vertex_102[0]))
        approx_102[1] = np.roll(approx_102[1], 2 * (len(approx_102[1]) - concave_vertex_102[1]))

        # add points of 102 to 3d points list
        list_3d_points.extend([rotate((0, 0), (70, 0), 12 * np.pi / 12), rotate((0, 0), (65, 5), 12 * np.pi / 12), rotate((0, 0), (98, 5), 12 * np.pi / 12), rotate((0, 0), (98, -5), 12 * np.pi / 12), rotate((0, 0), (65, -5), 12 * np.pi / 12)])
        list_3d_points.extend([rotate((0, 0), (70, 0), 4 * np.pi / 12), rotate((0, 0), (65, 5), 4 * np.pi / 12), rotate((0, 0), (98, 5), 4 * np.pi / 12), rotate((0, 0), (98, -5), 4 * np.pi / 12), rotate((0, 0), (65, -5), 4 * np.pi / 12)])

        # calculate the distance between their two points
        first = (points_102[0][0][0] - points_102[0][1][0]) ** 2 + (points_102[0][0][1] - points_102[0][1][1]) ** 2
        second = (points_102[1][0][0] - points_102[1][1][0]) ** 2 + (points_102[1][0][1] - points_102[1][1][1]) ** 2

        cv2.drawContours(actual_image, contours_102[0], 0, (128, 128, 128), 3)  # gray
        cv2.drawContours(actual_image, contours_102[1], 0, (128, 128, 128), 3)

        # if first distance is greater than the second one
        if first > second:

            # add points of 102 to 2d points list
            list_2d_points.extend(approx_102[1])
            list_2d_points.extend(approx_102[0])

        else:

            # add points of 102 to 2d points list
            list_2d_points.extend(approx_102[0])
            list_2d_points.extend(approx_102[1])

    # if there are two 110 in the image
    if len(points_110) == 2:

        # reorder array of points using the concave vertex in the first position
        approx_110[0] = np.roll(approx_110[0], 2 * (len(approx_110[0]) - concave_vertex_110[0]))
        approx_110[1] = np.roll(approx_110[1], 2 * (len(approx_110[1]) - concave_vertex_110[1]))

        # add points of 110 to 3d points list
        list_3d_points.extend([rotate((0, 0), (70, 0), 13 * np.pi / 12), rotate((0, 0), (65, 5), 13 * np.pi / 12), rotate((0, 0), (98, 5), 13 * np.pi / 12), rotate((0, 0), (98, -5), 13 * np.pi / 12), rotate((0, 0), (65, -5), 13 * np.pi / 12)])
        list_3d_points.extend([rotate((0, 0), (70, 0), 5 * np.pi / 12), rotate((0, 0), (65, 5), 5 * np.pi / 12), rotate((0, 0), (98, 5), 5 * np.pi / 12), rotate((0, 0), (98, -5), 5 * np.pi / 12), rotate((0, 0), (65, -5), 5 * np.pi / 12)])

        # calculate the distance between their two points
        first = (points_110[0][0][0] - points_110[0][1][0]) ** 2 + (points_110[0][0][1] - points_110[0][1][1]) ** 2
        second = (points_110[1][0][0] - points_110[1][1][0]) ** 2 + (points_110[1][0][1] - points_110[1][1][1]) ** 2

        cv2.drawContours(actual_image, contours_110[0], 0, (0, 255, 255), 3)  # yellow
        cv2.drawContours(actual_image, contours_110[1], 0, (0, 255, 255), 3)

        # if first distance is greater than the second one
        if first > second:

            # add points of 110 to 2d points list
            list_2d_points.extend(approx_110[1])
            list_2d_points.extend(approx_110[0])

        else:

            # add points of 110 to 2d points list
            list_2d_points.extend(approx_110[0])
            list_2d_points.extend(approx_110[1])

    # if there are two 112 in the image
    if len(points_112) == 2:

        # reorder array of points using the concave vertex in the first position
        approx_112[0] = np.roll(approx_112[0], 2 * (len(approx_112[0]) - concave_vertex_112[0]))
        approx_112[1] = np.roll(approx_112[1], 2 * (len(approx_112[1]) - concave_vertex_112[1]))

        # add points of 112 to 3d points list
        list_3d_points.extend([rotate((0, 0), (70, 0), 16 * np.pi / 12), rotate((0, 0), (65, 5), 16 * np.pi / 12), rotate((0, 0), (98, 5), 16 * np.pi / 12), rotate((0, 0), (98, -5), 16 * np.pi / 12), rotate((0, 0), (65, -5), 16 * np.pi / 12)])
        list_3d_points.extend([rotate((0, 0), (70, 0), 8 * np.pi / 12), rotate((0, 0), (65, 5), 8 * np.pi / 12), rotate((0, 0), (98, 5), 8 * np.pi / 12), rotate((0, 0), (98, -5), 8 * np.pi / 12), rotate((0, 0), (65, -5), 8 * np.pi / 12)])

        # calculate the distance between their two points
        first = (points_112[0][0][0] - points_112[0][1][0]) ** 2 + (points_112[0][0][1] - points_112[0][1][1]) ** 2
        second = (points_112[1][0][0] - points_112[1][1][0]) ** 2 + (points_112[1][0][1] - points_112[1][1][1]) ** 2

        cv2.drawContours(actual_image, contours_112[0], 0, (203, 192, 255), 3)  # pink
        cv2.drawContours(actual_image, contours_112[1], 0, (203, 192, 255), 3)

        # if first distance is greater than the second one
        if first > second:

            # add points of 112 to 2d points list
            list_2d_points.extend(approx_112[1])
            list_2d_points.extend(approx_112[0])

        else:

            # add points of 112 to 2d points list
            list_2d_points.extend(approx_112[0])
            list_2d_points.extend(approx_112[1])

    # if there are two 211 in the image
    if len(points_211) == 2:

        # reorder array of points using the concave vertex in the first position
        approx_211[0] = np.roll(approx_211[0], 2 * (len(approx_211[0]) - concave_vertex_211[0]))
        approx_211[1] = np.roll(approx_211[1], 2 * (len(approx_211[1]) - concave_vertex_211[1]))

        # add points of 211 to 3d points list
        list_3d_points.extend([rotate((0, 0), (70, 0), -2 * np.pi / 12), rotate((0, 0), (65, 5), -2 * np.pi / 12), rotate((0, 0), (98, 5), -2 * np.pi / 12), rotate((0, 0), (98, -5), -2 * np.pi / 12), rotate((0, 0), (65, -5), -2 * np.pi / 12)])
        list_3d_points.extend([rotate((0, 0), (70, 0), -1 * np.pi / 12), rotate((0, 0), (65, 5), -1 * np.pi / 12), rotate((0, 0), (98, 5), -1 * np.pi / 12), rotate((0, 0), (98, -5), -1 * np.pi / 12), rotate((0, 0), (65, -5), -1 * np.pi / 12)])

        # calculate the distance between their two points
        first = (points_211[0][len(points_211[0]) - 2][0] - points_211[0][len(points_211[0]) - 1][0]) ** 2 + (points_211[0][len(points_211[0]) - 2][1] - points_211[0][len(points_211[0]) - 1][1]) ** 2
        second = (points_211[1][len(points_211[1]) - 2][0] - points_211[1][len(points_211[1]) - 1][0]) ** 2 + (points_211[1][len(points_211[1]) - 2][1] - points_211[1][len(points_211[1]) - 1][1]) ** 2

        cv2.drawContours(actual_image, contours_211[0], 0, (0, 102, 255), 3)  # orange
        cv2.drawContours(actual_image, contours_211[1], 0, (0, 102, 255), 3)

        # if first distance is greater than the second one
        if first > second:

            # add points of 211 to 2d points list
            list_2d_points.extend(approx_211[1])
            list_2d_points.extend(approx_211[0])

        else:

            # add points of 211 to 2d points list
            list_2d_points.extend(approx_211[0])
            list_2d_points.extend(approx_211[1])

    # if there are two 201 in the image
    if len(points_201) == 2:

        # reorder array of points using the concave vertex in the first position
        approx_201[0] = np.roll(approx_201[0], 2 * (len(approx_201[0]) - concave_vertex_201[0]))
        approx_201[1] = np.roll(approx_201[1], 2 * (len(approx_201[1]) - concave_vertex_201[1]))

        # add points of 201 to 3d points list
        list_3d_points.extend([rotate((0, 0), (70, 0), -6 * np.pi / 12), rotate((0, 0), (65, 5), -6 * np.pi / 12), rotate((0, 0), (98, 5), -6 * np.pi / 12), rotate((0, 0), (98, -5), -6 * np.pi / 12), rotate((0, 0), (65, -5), -6 * np.pi / 12)])
        list_3d_points.extend([rotate((0, 0), (70, 0), -5 * np.pi / 12), rotate((0, 0), (65, 5), -5 * np.pi / 12), rotate((0, 0), (98, 5), -5 * np.pi / 12), rotate((0, 0), (98, -5), -5 * np.pi / 12), rotate((0, 0), (65, -5), -5 * np.pi / 12)])

        # calculate the distance between their two points
        first = (points_201[0][len(points_201[0]) - 2][0] - points_201[0][len(points_201[0]) - 1][0]) ** 2 + (points_201[0][len(points_201[0]) - 2][1] - points_201[0][len(points_201[0]) - 1][1]) ** 2
        second = (points_201[1][len(points_201[1]) - 2][0] - points_201[1][len(points_201[1]) - 1][0]) ** 2 + (points_201[1][len(points_201[1]) - 2][1] - points_201[1][len(points_201[1]) - 1][1]) ** 2

        cv2.drawContours(actual_image, contours_201[0], 0, (0, 0, 0), 3)  # black
        cv2.drawContours(actual_image, contours_201[1], 0, (0, 0, 0), 3)

        # if first distance is greater than the second one
        if first > second:

            # add points of 201 to 2d points list
            list_2d_points.extend(approx_201[1])
            list_2d_points.extend(approx_201[0])

        else:

            # add points of 201 to 2d points list
            list_2d_points.extend(approx_201[0])
            list_2d_points.extend(approx_201[1])

    # if there are four 111 in the image
    if len(points_111) == 4:

        # reorder array of points using the concave vertex in the first position
        approx_111[0] = np.roll(approx_111[0], 2 * (len(approx_111[0]) - concave_vertex_111[0]))
        approx_111[1] = np.roll(approx_111[1], 2 * (len(approx_111[1]) - concave_vertex_111[1]))
        approx_111[2] = np.roll(approx_111[2], 2 * (len(approx_111[2]) - concave_vertex_111[2]))
        approx_111[3] = np.roll(approx_111[3], 2 * (len(approx_111[3]) - concave_vertex_111[3]))

        # add points of 111 to 3d points list
        list_3d_points.extend([rotate((0, 0), (70, 0), 14 * np.pi / 12), rotate((0, 0), (65, 5), 14 * np.pi / 12), rotate((0, 0), (98, 5), 14 * np.pi / 12), rotate((0, 0), (98, -5), 14 * np.pi / 12), rotate((0, 0), (65, -5), 14 * np.pi / 12)])
        list_3d_points.extend([rotate((0, 0), (70, 0), 15 * np.pi / 12), rotate((0, 0), (65, 5), 15 * np.pi / 12), rotate((0, 0), (98, 5), 15 * np.pi / 12), rotate((0, 0), (98, -5), 15 * np.pi / 12), rotate((0, 0), (65, -5), 15 * np.pi / 12)])
        list_3d_points.extend([rotate((0, 0), (70, 0), 6 * np.pi / 12), rotate((0, 0), (65, 5), 6 * np.pi / 12), rotate((0, 0), (98, 5), 6 * np.pi / 12), rotate((0, 0), (98, -5), 6 * np.pi / 12), rotate((0, 0), (65, -5), 6 * np.pi / 12)])
        list_3d_points.extend([rotate((0, 0), (70, 0), 7 * np.pi / 12), rotate((0, 0), (65, 5), 7 * np.pi / 12), rotate((0, 0), (98, 5), 7 * np.pi / 12), rotate((0, 0), (98, -5), 7 * np.pi / 12), rotate((0, 0), (65, -5), 7 * np.pi / 12)])

        # calculate the distance between their four points
        first_outer = (points_111[0][0][0] - points_111[0][1][0]) ** 2 + (points_111[0][0][1] - points_111[0][1][1]) ** 2
        second_outer = (points_111[1][0][0] - points_111[1][1][0]) ** 2 + (points_111[1][0][1] - points_111[1][1][1]) ** 2
        third_outer = (points_111[2][0][0] - points_111[2][1][0]) ** 2 + (points_111[2][0][1] - points_111[2][1][1]) ** 2
        fourth_outer = (points_111[3][0][0] - points_111[3][1][0]) ** 2 + (points_111[3][0][1] - points_111[3][1][1]) ** 2

        first_inner = (points_111[0][len(points_111[0]) - 2][0] - points_111[0][len(points_111[0]) - 1][0]) ** 2 + (points_111[0][len(points_111[0]) - 2][1] - points_111[0][len(points_111[0]) - 1][1]) ** 2
        second_inner = (points_111[1][len(points_111[1]) - 2][0] - points_111[1][len(points_111[1]) - 1][0]) ** 2 + (points_111[1][len(points_111[1]) - 2][1] - points_111[1][len(points_111[1]) - 1][1]) ** 2
        third_inner = (points_111[2][len(points_111[2]) - 2][0] - points_111[2][len(points_111[2]) - 1][0]) ** 2 + (points_111[2][len(points_111[2]) - 2][1] - points_111[2][len(points_111[2]) - 1][1]) ** 2
        fourth_inner = (points_111[3][len(points_111[3]) - 2][0] - points_111[3][len(points_111[3]) - 1][0]) ** 2 + (points_111[3][len(points_111[3]) - 2][1] - points_111[3][len(points_111[3]) - 1][1]) ** 2

        # create and sort array with outer margin with an original copy
        outer_margin = np.array([first_outer, second_outer, third_outer, fourth_outer])
        outer_margin_sort = outer_margin.copy()
        outer_margin_sort.sort()

        # create array of inner margin
        inner_margin = np.array([first_inner, second_inner, third_inner, fourth_inner])

        # indexes for outer margin
        outer_1 = 0
        outer_2 = 0
        outer_3 = 0
        outer_4 = 0

        # get the right index for the outer margins
        for i in range(0, len(outer_margin)):

            if outer_margin_sort[0] == outer_margin[i]:
                outer_1 = i

            if outer_margin_sort[1] == outer_margin[i]:
                outer_2 = i

            if outer_margin_sort[2] == outer_margin[i]:
                outer_3 = i

            if outer_margin_sort[3] == outer_margin[i]:
                outer_4 = i

        cv2.drawContours(actual_image, contours_111[outer_1], 0, (255, 255, 255), 3)  # white
        cv2.drawContours(actual_image, contours_111[outer_2], 0, (255, 255, 255), 3)
        cv2.drawContours(actual_image, contours_111[outer_3], 0, (255, 255, 255), 3)
        cv2.drawContours(actual_image, contours_111[outer_4], 0, (255, 255, 255), 3)

        # check on inner margins
        if inner_margin[outer_1] > inner_margin[outer_2]:

            # add points of 111 to 2d points list
            list_2d_points.extend(approx_111[outer_2])
            list_2d_points.extend(approx_111[outer_1])

        else:

            # add points of 111 to 2d points list
            list_2d_points.extend(approx_111[outer_1])
            list_2d_points.extend(approx_111[outer_2])

        # check on inner margins
        if inner_margin[outer_3] > inner_margin[outer_4]:

            # add points of 111 to 2d points list
            list_2d_points.extend(approx_111[outer_4])
            list_2d_points.extend(approx_111[outer_3])

        else:

            # add points of 111 to 2d points list
            list_2d_points.extend(approx_111[outer_3])
            list_2d_points.extend(approx_111[outer_4])

    # if there are four 101 in the image
    if len(points_101) == 4:

        # reorder array of points using the concave vertex in the first position
        approx_101[0] = np.roll(approx_101[0], 2 * (len(approx_101[0]) - concave_vertex_101[0]))
        approx_101[1] = np.roll(approx_101[1], 2 * (len(approx_101[1]) - concave_vertex_101[1]))
        approx_101[2] = np.roll(approx_101[2], 2 * (len(approx_101[2]) - concave_vertex_101[2]))
        approx_101[3] = np.roll(approx_101[3], 2 * (len(approx_101[3]) - concave_vertex_101[3]))

        # add points of 101 to 3d points list
        list_3d_points.extend([rotate((0, 0), (70, 0), 10 * np.pi / 12), rotate((0, 0), (65, 5), 10 * np.pi / 12), rotate((0, 0), (98, 5), 10 * np.pi / 12), rotate((0, 0), (98, -5), 10 * np.pi / 12), rotate((0, 0), (65, -5), 10 * np.pi / 12)])
        list_3d_points.extend([rotate((0, 0), (70, 0), 11 * np.pi / 12), rotate((0, 0), (65, 5), 11 * np.pi / 12), rotate((0, 0), (98, 5), 11 * np.pi / 12), rotate((0, 0), (98, -5), 11 * np.pi / 12), rotate((0, 0), (65, -5), 11 * np.pi / 12)])
        list_3d_points.extend([rotate((0, 0), (70, 0), 2 * np.pi / 12), rotate((0, 0), (65, 5), 2 * np.pi / 12), rotate((0, 0), (98, 5), 2 * np.pi / 12), rotate((0, 0), (98, -5), 2 * np.pi / 12), rotate((0, 0), (65, -5), 2 * np.pi / 12)])
        list_3d_points.extend([rotate((0, 0), (70, 0), 3 * np.pi / 12), rotate((0, 0), (65, 5), 3 * np.pi / 12), rotate((0, 0), (98, 5), 3 * np.pi / 12), rotate((0, 0), (98, -5), 3 * np.pi / 12), rotate((0, 0), (65, -5), 3 * np.pi / 12)])

        # calculate the distance between their four points
        first_outer = (points_101[0][0][0] - points_101[0][1][0]) ** 2 + (points_101[0][0][1] - points_101[0][1][1]) ** 2
        second_outer = (points_101[1][0][0] - points_101[1][1][0]) ** 2 + (points_101[1][0][1] - points_101[1][1][1]) ** 2
        third_outer = (points_101[2][0][0] - points_101[2][1][0]) ** 2 + (points_101[2][0][1] - points_101[2][1][1]) ** 2
        fourth_outer = (points_101[3][0][0] - points_101[3][1][0]) ** 2 + (points_101[3][0][1] - points_101[3][1][1]) ** 2

        first_inner = (points_101[0][len(points_101[0]) - 2][0] - points_101[0][len(points_101[0]) - 1][0]) ** 2 + (points_101[0][len(points_101[0]) - 2][1] - points_101[0][len(points_101[0]) - 1][1]) ** 2
        second_inner = (points_101[1][len(points_101[1]) - 2][0] - points_101[1][len(points_101[1]) - 1][0]) ** 2 + (points_101[1][len(points_101[1]) - 2][1] - points_101[1][len(points_101[1]) - 1][1]) ** 2
        third_inner = (points_101[2][len(points_101[2]) - 2][0] - points_101[2][len(points_101[2]) - 1][0]) ** 2 + (points_101[2][len(points_101[2]) - 2][1] - points_101[2][len(points_101[2]) - 1][1]) ** 2
        fourth_inner = (points_101[3][len(points_101[3]) - 2][0] - points_101[3][len(points_101[3]) - 1][0]) ** 2 + (points_101[3][len(points_101[3]) - 2][1] - points_101[3][len(points_101[3]) - 1][1]) ** 2

        # create and sort array with outer margin with an original copy
        outer_margin = np.array([first_outer, second_outer, third_outer, fourth_outer])
        outer_margin_sort = outer_margin.copy()
        outer_margin_sort.sort()

        # create array of inner margin
        inner_margin = np.array([first_inner, second_inner, third_inner, fourth_inner])

        # indexes for outer margin
        outer_1 = 0
        outer_2 = 0
        outer_3 = 0
        outer_4 = 0

        # get the right index for the outer margins
        for i in range(0, len(outer_margin)):

            if outer_margin_sort[0] == outer_margin[i]:
                outer_1 = i

            if outer_margin_sort[1] == outer_margin[i]:
                outer_2 = i

            if outer_margin_sort[2] == outer_margin[i]:
                outer_3 = i

            if outer_margin_sort[3] == outer_margin[i]:
                outer_4 = i

        cv2.drawContours(actual_image, contours_101[outer_1], 0, (127, 255, 212), 3)  # light blue
        cv2.drawContours(actual_image, contours_101[outer_2], 0, (127, 255, 212), 3)
        cv2.drawContours(actual_image, contours_101[outer_3], 0, (127, 255, 212), 3)
        cv2.drawContours(actual_image, contours_101[outer_4], 0, (127, 255, 212), 3)

        # check on inner margins
        if inner_margin[outer_1] > inner_margin[outer_2]:

            # add points of 101 to 2d points list
            list_2d_points.extend(approx_101[outer_2])
            list_2d_points.extend(approx_101[outer_1])

        else:

            # add points of 101 to 2d points list
            list_2d_points.extend(approx_101[outer_1])
            list_2d_points.extend(approx_101[outer_2])

        # check on inner margins
        if inner_margin[outer_3] > inner_margin[outer_4]:

            # add points of 101 to 2d points list
            list_2d_points.extend(approx_101[outer_4])
            list_2d_points.extend(approx_101[outer_3])

        else:

            # add points of 101 to 2d points list
            list_2d_points.extend(approx_101[outer_3])
            list_2d_points.extend(approx_101[outer_4])

    # convert to float the list of 2d points and simplify the structure
    list_2d_points = np.array(list_2d_points).squeeze().astype("float32")

    # convert to float the list of 3d points and simplify the structure
    list_3d_points = np.array(list_3d_points).squeeze().astype("float32")

    # save actual image with border and polygon recognition in polygon border folder
    cv2.imwrite(image.replace("uncalibrated", "polygonborders"), actual_image)

    # ---------------------------------------------------------------------------------------------------- FINISH LIST OF 2D AND 3D POINTS FOR THE NEXT PROJECTION PROCESS

    # ---------------------------------------------------------------------------------------------------- PROJECTION POINT AND PLUS TEXT

    # read again two actual original image to project into these the contours of each polygon plus the binary value
    img = cv2.imread(image, cv2.IMREAD_COLOR)
    img_with_text = cv2.imread(image, cv2.IMREAD_COLOR)

    # get height and width to use them then with the new camera matrix calculation
    h, w = img.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(K, dist, (w, h), 1, (w, h))

    # solvepnp function with the reference of 2d and 3d points calculated before
    success, rotation_vector, translation_vector = cv2.solvePnP(list_3d_points, list_2d_points, newcameramtx, dist, cv2.SOLVEPNP_IPPE)

    # for each black polygon
    for i in range(0, 24):

        # list for the points of each polygon
        coordinates = []

        # project the vertices using the rotate function
        nose_end_point2D, jacobian = cv2.projectPoints(np.array([rotate((0, 0), (70, 0), i * np.pi / 12)]).squeeze().astype("float32"), rotation_vector, translation_vector, newcameramtx, dist)
        coordinates.extend(nose_end_point2D[0])
        nose_end_point2D, jacobian = cv2.projectPoints(np.array([rotate((0, 0), (65, 5), i * np.pi / 12)]).squeeze().astype("float32"), rotation_vector, translation_vector, newcameramtx, dist)
        coordinates.extend(nose_end_point2D[0])
        nose_end_point2D, jacobian = cv2.projectPoints(np.array([rotate((0, 0), (98, 5), i * np.pi / 12)]).squeeze().astype("float32"), rotation_vector, translation_vector, newcameramtx, dist)
        coordinates.extend(nose_end_point2D[0])
        nose_end_point2D, jacobian = cv2.projectPoints(np.array([rotate((0, 0), (98, -5), i * np.pi / 12)]).squeeze().astype("float32"), rotation_vector, translation_vector, newcameramtx, dist)
        coordinates.extend(nose_end_point2D[0])
        nose_end_point2D, jacobian = cv2.projectPoints(np.array([rotate((0, 0), (65, -5), i * np.pi / 12)]).squeeze().astype("float32"), rotation_vector, translation_vector, newcameramtx, dist)
        coordinates.extend(nose_end_point2D[0])

        # list to store the central points of the white circles into each polygon
        list_central_points = []

        # project the center of circles into each polygon
        nose_end_point2D, jacobian = cv2.projectPoints(np.array([rotate((0, 0), (75, 0), i * np.pi / 12)]).squeeze().astype("float32"), rotation_vector, translation_vector, newcameramtx, dist)
        list_central_points.extend(nose_end_point2D[0])
        nose_end_point2D, jacobian = cv2.projectPoints(np.array([rotate((0, 0), (79.5, 0), i * np.pi / 12)]).squeeze().astype("float32"), rotation_vector, translation_vector, newcameramtx, dist)
        list_central_points.extend(nose_end_point2D[0])
        nose_end_point2D, jacobian = cv2.projectPoints(np.array([rotate((0, 0), (84, 0), i * np.pi / 12)]).squeeze().astype("float32"), rotation_vector, translation_vector, newcameramtx, dist)
        list_central_points.extend(nose_end_point2D[0])
        nose_end_point2D, jacobian = cv2.projectPoints(np.array([rotate((0, 0), (88.5, 0), i * np.pi / 12)]).squeeze().astype("float32"), rotation_vector, translation_vector, newcameramtx, dist)
        list_central_points.extend(nose_end_point2D[0])
        nose_end_point2D, jacobian = cv2.projectPoints(np.array([rotate((0, 0), (93, 0), i * np.pi / 12)]).squeeze().astype("float32"), rotation_vector, translation_vector, newcameramtx, dist)
        list_central_points.extend(nose_end_point2D[0])

        # create a copy of coordinates points to draw then all the points
        list_of_points = coordinates.copy()

        # transform the coordinates array to use it in the next function
        coordinates = np.array(coordinates, dtype=np.int32).squeeze().reshape((-1, 1, 2))

        # draw the contour of the actual polygon with blue line
        img = cv2.polylines(img, [coordinates], True, (255, 0, 0), 3)
        img_with_text = cv2.polylines(img_with_text, [coordinates], True, (255, 0, 0), 3)

        # for each point vertex of the polygon
        for point in list_of_points:
            # draw a circle
            img = cv2.circle(img, (round(point[0]), round(point[1])), 2, (0, 0, 255), -1)
            img_with_text = cv2.circle(img_with_text, (round(point[0]), round(point[1])), 2, (0, 0, 255), -1)

        # for each center point of the circles into each polygon
        for point in list_central_points:
            # draw a circle
            img = cv2.circle(img, (round(point[0]), round(point[1])), 2, (0, 0, 255), -1)
            img_with_text = cv2.circle(img_with_text, (round(point[0]), round(point[1])), 2, (0, 0, 255), -1)

        # obtain the projection of the point between the points c and d to write in the center the correct number
        nose_end_point2D, jacobian = cv2.projectPoints(np.array([rotate((0, 0), (98, 0), i * np.pi / 12)]).squeeze().astype("float32"), rotation_vector, translation_vector, newcameramtx, dist)

        # write the binary value in decimal type in each polygon
        img_with_text = cv2.putText(img_with_text, str((24 - i) % 24), (round(nose_end_point2D[0][0][0]), round(nose_end_point2D[0][0][1])), cv2.FONT_ITALIC, 1, (0, 0, 255), 6, cv2.LINE_4, False)

    # save the actual image in the projection points folder and projection points with text folder
    cv2.imwrite(image.replace("uncalibrated", "projectionpoints"), img)
    cv2.imwrite(image.replace("uncalibrated", "projectionpointswithtext"), img_with_text)

    # ---------------------------------------------------------------------------------------------------- FINISH PROJECTION POINT AND PLUS TEXT

    # ---------------------------------------------------------------------------------------------------- PROJECTION POINTS IN OR OUT THE OBJECT

    # read the actual image but in the background removed version
    img = cv2.imread(image.replace("uncalibrated", "backgroundremoved"), cv2.IMREAD_COLOR)

    # for each dimension of the cube
    for i in range(0, 70):
        for j in range(0, 70):
            for k in range(0, 60):

                # project the actual point of the cube but in (-70;+70),(-70;+70),(+80;+200) cube using a smaller set of dimension to get a fast algorithm
                nose_end_point2D, jacobian = cv2.projectPoints(np.array([((i) - 35) * 2, ((j) - 35) * 2, 80 + k * 2]).squeeze().astype("float32"), rotation_vector, translation_vector, newcameramtx, dist)

                # simplify and round the structure of the coordinates
                nose_end_point2D = nose_end_point2D.squeeze()
                nose_end_point2D = [round(nose_end_point2D[0]), round(nose_end_point2D[1])]

                # if the coordinates are into the dimensions of the image 1080x1920
                if 0 <= nose_end_point2D[1] < 1080 and 0 <= nose_end_point2D[0] < 1920:

                    # if the actual coordinates cube value is not checked or inside the object
                    if cube[i][j][k] == 0 or cube[i][j][k] == 1:

                        # check if the actual position is 000 that in rgb vallue means black because we set from background deletion algorithm the object as a black obj
                        if np.array_equal(img[nose_end_point2D[1]][nose_end_point2D[0]], [0, 0, 0]):

                            # if the actual pixel is black means that is into the object
                            cube[i][j][k] = 1

                        # the actual pixel is not black
                        else:

                            # remove the actual pixel from the cube
                            cube[i][j][k] = 2

    # ---------------------------------------------------------------------------------------------------- FINISH PROJECTION POINTS IN OR OUT THE OBJECT

# ---------------------------------------------------------------------------------------------------- CREATION OF PLY FILE

# variable used to store the number of vertices
contVertices = 0

# for each dimension of the cube
for i in range(0, 70):
    for j in range(0, 70):
        for k in range(0, 60):

            # if the actual position is inside the object
            if cube[i][j][k] == 1:
                # add 8 vertices in the counter because a cube has 8 vertices
                contVertices = contVertices + 8

# variable used to store the number of faces
contFaces = 0

# for each dimension of the cube
for i in range(0, 70):
    for j in range(0, 70):
        for k in range(0, 60):

            # if the actual position is inside the object
            if cube[i][j][k] == 1:
                # add 6 faces because a cube has 6 faces
                contFaces = contFaces + 6

# create a ply file in the output folder of the actual video
plyFile = open("framesVideo" + str(videoToElaborate) + "/output/file_more_accurate.ply", "w+")

# write into the ply file the whole header
plyFile.write("ply\n")
plyFile.write("format ascii 1.0\n")
plyFile.write("element vertex " + str(contVertices) + "\n")
plyFile.write("property float x\n")
plyFile.write("property float y\n")
plyFile.write("property float z\n")
plyFile.write("element face " + str(contFaces) + "\n")
plyFile.write("property list uint8 int vertex_indices\n")
plyFile.write("property uchar red\n")
plyFile.write("property uchar green\n")
plyFile.write("property uchar blue\n")
plyFile.write("end_header\n")

# for each dimension of the cube
for i in range(0, 70):
    for j in range(0, 70):
        for k in range(0, 60):

            # if the actual position is inside the object
            if cube[i][j][k] == 1:
                # write into the ply file the 8 vertices
                plyFile.write(str(0 + i) + " " + str(0 + j) + " " + str(0 + k) + "\n")
                plyFile.write(str(2 + i) + " " + str(0 + j) + " " + str(0 + k) + "\n")
                plyFile.write(str(2 + i) + " " + str(2 + j) + " " + str(0 + k) + "\n")
                plyFile.write(str(0 + i) + " " + str(2 + j) + " " + str(0 + k) + "\n")
                plyFile.write(str(0 + i) + " " + str(0 + j) + " " + str(1 + k) + "\n")
                plyFile.write(str(2 + i) + " " + str(0 + j) + " " + str(1 + k) + "\n")
                plyFile.write(str(2 + i) + " " + str(2 + j) + " " + str(1 + k) + "\n")
                plyFile.write(str(0 + i) + " " + str(2 + j) + " " + str(1 + k) + "\n")

# variable used to store the sum of vertices of cube
sumVerticesCube = 0

# for each dimension of the cube
for i in range(0, 70):
    for j in range(0, 70):
        for k in range(0, 60):

            # if the actual position is inside the object
            if cube[i][j][k] == 1:
                # create all faces of the actual cube
                plyFile.write("4 " + str(sumVerticesCube + 0) + " " + str(sumVerticesCube + 1) + " " + str(sumVerticesCube + 2) + " " + str(sumVerticesCube + 3) + " 255 0 0\n")
                plyFile.write("4 " + str(sumVerticesCube + 4) + " " + str(sumVerticesCube + 5) + " " + str(sumVerticesCube + 6) + " " + str(sumVerticesCube + 7) + " 255 0 0\n")
                plyFile.write("4 " + str(sumVerticesCube + 1) + " " + str(sumVerticesCube + 2) + " " + str(sumVerticesCube + 6) + " " + str(sumVerticesCube + 5) + " 255 0 0\n")
                plyFile.write("4 " + str(sumVerticesCube + 2) + " " + str(sumVerticesCube + 3) + " " + str(sumVerticesCube + 7) + " " + str(sumVerticesCube + 6) + " 255 0 0\n")
                plyFile.write("4 " + str(sumVerticesCube + 3) + " " + str(sumVerticesCube + 0) + " " + str(sumVerticesCube + 4) + " " + str(sumVerticesCube + 7) + " 255 0 0\n")
                plyFile.write("4 " + str(sumVerticesCube + 0) + " " + str(sumVerticesCube + 1) + " " + str(sumVerticesCube + 5) + " " + str(sumVerticesCube + 4) + " 255 0 0\n")

                # shift the faces of the actual cube using the 8 vertices for each cube
                sumVerticesCube = sumVerticesCube + 8

# close the ply file
plyFile.close()

